In [6]:
%AddJar http://cedric.cnam.fr/~ferecatu/RCP216/tp/tptexte/lsa.jar

Starting download from http://cedric.cnam.fr/~ferecatu/RCP216/tp/tptexte/lsa.jar
Finished download of lsa.jar


In [7]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd

import scala.xml._

import org.apache.hadoop.io.{ Text, LongWritable }
import org.apache.hadoop.conf.Configuration

import com.cloudera.datascience.common.XmlInputFormat
import com.cloudera.datascience.lsa.ParseWikipedia._
import com.cloudera.datascience.lsa.RunLSA._

import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix => BDenseMatrix, DenseVector => BDenseVector, SparseVector => BSparseVector, Vector => BVector}
import org.apache.spark.mllib.feature.{Word2Vec, Word2VecModel}
import org.apache.spark.mllib.feature.StandardScaler



import java.io.StringReader
import edu.stanford.nlp.ling.CoreLabel;
import edu.stanford.nlp.ling.HasWord;
import edu.stanford.nlp.process.CoreLabelTokenFactory;
import edu.stanford.nlp.process.DocumentPreprocessor;
import edu.stanford.nlp.process.PTBTokenizer;

import java.sql.Timestamp
import java.text.SimpleDateFormat

import org.apache.spark.mllib.clustering.{ KMeans, KMeansModel }
import org.apache.spark.mllib.util.KMeansDataGenerator

import scala.collection.mutable.HashMap
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD

import breeze.linalg.{DenseMatrix=>BDenseMatrix, DenseVector=>BDenseVector, SparseVector=>BSparseVector, Vector=>BVector}


In [8]:
def toBreeze(v:Vector) = BVector(v.toArray)
def fromBreeze(bv:BVector[Double]) = Vectors.dense(bv.toArray)
def add(v1:Vector, v2:Vector) = fromBreeze(toBreeze(v1) + toBreeze(v2))
def scalarMultiply(a:Double, v:Vector) = fromBreeze(a * toBreeze(v))

def stackVectors(v1:Vector, v2:Vector) = {
var v3 = Vectors.zeros(v1.size+v2.size)
    for (i <- 0 until v1.size) {
      BVector(v3.toArray)(i) = v1(i);
    }
    for (i <- 0 until v2.size) {
      BVector(v3.toArray)(v1.size+i) = v2(i);
    }
}

// SÃ©paration du fichier XML en un RDD oÃ¹ chaque Ã©lÃ©ment est un article
// Retourne un RDD de String Ã  partir du fichier "path"
def loadArticle(sc: SparkContext, path: String): RDD[String] = {
@transient val conf = new Configuration()
conf.set(XmlInputFormat.START_TAG_KEY, "<document>")
conf.set(XmlInputFormat.END_TAG_KEY, "</document>")
val in = sc.newAPIHadoopFile(path, classOf[XmlInputFormat], classOf[LongWritable], classOf[Text], conf)
in.map(line => line._2.toString)
}


// Pour un élément XML de type "document",
//   - on extrait le champ "date"
//   - on parse la chaÃ®ne de caractÃ¨re au format yyyy-MM-dd HH:mm:ss
//   - on retourne un Timestamp
def extractDate(elem: scala.xml.Elem): java.sql.Timestamp = {
    val dn: scala.xml.NodeSeq = elem \\ "date"
    val x: String = dn.text
    // d'aprÃ¨s l'exemple 2011-05-18 16:30:35
    val format = new SimpleDateFormat("yyyy-MM-dd")
    if (x == "")
      return null
    else {
      val d = format.parse(x.toString());
      val t = new Timestamp(d.getTime());
      return t
    }
}

// Pour un élément XML de type "document",
//   - on extrait le champ #field
def extractString(elem: scala.xml.Elem, field: String): String = {
    val dn: scala.xml.NodeSeq = elem \\ field
    val x: String = dn.text
    return x
}

def extractInt(elem: scala.xml.Elem, field: String): Int = {
    val dn: scala.xml.NodeSeq = elem \\ field
    val x: Int = dn.text.toInt
    return x
}

def extractAll(elem: scala.xml.Elem, whatText: String = "text"): (Int, java.sql.Timestamp, String) = {
    return (extractInt(elem,"docid"), extractDate(elem), extractString(elem,whatText))
}

def extractText(elem: scala.xml.Elem): String = {
    return (extractString(elem,"title") + " " + extractString(elem,"text"))
}

// NÃ©cessaire, car le type java.sql.Timestamp n'est pas ordonnÃ© par dÃ©faut (étonnant...)
implicit def ordered: Ordering[java.sql.Timestamp] = new Ordering[java.sql.Timestamp] {
def compare(x: java.sql.Timestamp, y: java.sql.Timestamp): Int = x compareTo y
}

def hasLetters(str: String): Boolean = {
// While loop for high performance
var i = 0
while (i < str.length) {
  if (Character.isLetter(str.charAt(i))) {
    return true
  }
  i += 1
}
false
}

toBreeze: (v: org.apache.spark.mllib.linalg.Vector)breeze.linalg.Vector[Double]
fromBreeze: (bv: breeze.linalg.Vector[Double])org.apache.spark.mllib.linalg.Vector
add: (v1: org.apache.spark.mllib.linalg.Vector, v2: org.apache.spark.mllib.linalg.Vector)org.apache.spark.mllib.linalg.Vector
scalarMultiply: (a: Double, v: org.apache.spark.mllib.linalg.Vector)org.apache.spark.mllib.linalg.Vector
stackVectors: (v1: org.apache.spark.mllib.linalg.Vector, v2: org.apache.spark.mllib.linalg.Vector)Unit
loadArticle: (sc: org.apache.spark.SparkContext, path: String)org.apache.spark.rdd.RDD[String]
extractDate: (elem: scala.xml.Elem)java.sql.Timestamp
extractString: (elem: scala.xml.Elem, field: String)String
extractInt: (elem: scala.xml.Elem, field: String)Int
extractAll: (elem: scala.xml.Elem, what...


In [9]:
    val stopwords = sc.textFile("stopwords.txt").collect.toArray.toSet
    val stopwordsBroadcast = sc.broadcast(stopwords).value

stopwords = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...


Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...

In [12]:
import org.apache.spark.mllib.feature.Word2VecModel
// lire le Word2VecModel
val w2vModel = Word2VecModel.load(sc, "w2vModel")

w2vModel = org.apache.spark.mllib.feature.Word2VecModel@f6e9ec5


org.apache.spark.mllib.feature.Word2VecModel@f6e9ec5

In [13]:
val conf = new SparkConf().setAppName("NEWS").setMaster("local[*]").set("spark.driver.memory", "4g")



conf.set("spark.hadoop.validateOutputSpecs", "false")
//conf.set("spark.driver.allowMultipleContexts", "true")
//conf.setMaster("local[*]")
//conf.set("spark.executor.memory", MAX_MEMORY)
//conf.set("spark.driver.memory", MAX_MEMORY)
//conf.set("spark.driver.maxResultSize", MAX_MEMORY)
val sc = new SparkContext(conf)


val hadoopConf = new org.apache.hadoop.conf.Configuration()

Name: org.apache.spark.SparkException
Message: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.sql.SparkSession$Builder.getOrCreate(SparkSession.scala:926)
org.apache.toree.kernel.api.Kernel.sparkSession(Kernel.scala:444)
org.apache.toree.kernel.api.Kernel.sparkContext(Kernel.scala:449)
org.apache.toree.kernel.api.Kernel$$anonfun$addJars$3.apply(Kernel.scala:80)
org.apache.toree.kernel.api.Kernel$$anonfun$addJars$3.apply(Kernel.scala:80)
scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
org.apache.toree.kernel.api.Kernel.addJars(Kernel.scala:80)
org.apache.toree.magic.builtin.AddJar.execute(AddJar.scala:181)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl

In [14]:
    var textToExtract = "text";
    var useW2Vec = true;
    var weightTfIdf = true;
    println ("*****");

*****


textToExtract = text
useW2Vec = true
weightTfIdf = true


lastException: Throwable = null


true

In [43]:
    val news_raw = loadArticle(sc, "articles1.xml")/*.sample(false,0.01)*/
    val news_xml: RDD[Elem] = news_raw.map(XML.loadString)

nysk_raw = MapPartitionsRDD[126] at map at <console>:99
nysk_xml = MapPartitionsRDD[127] at map at <console>:115


MapPartitionsRDD[127] at map at <console>:115

In [27]:
val articles1 = spark.read.format("csv").option("header","true").option("sep",";").load("articles.csv")

articles1 = [docid: string, title: string ... 6 more fields]


[docid: string, title: string ... 6 more fields]

In [45]:
articles1.show(5)


+-----+--------------------+--------------+--------------------+----------+----+-----+--------------------+
|docid|               title|   publication|              author|      date|year|month|                text|
+-----+--------------------+--------------+--------------------+----------+----+-----+--------------------+
|17283|House Republicans...|New York Times|          Carl Hulse|31/12/2016|2016|   12|WASHINGTON  �   C...|
|17291|First, a Mixtape....|New York Times|    Katherine Rosman|31/12/2016|2016|   12|Just how   is Hil...|
|17292|Calling on Angels...|New York Times|         Andy Newman|31/12/2016|2016|   12|Angels are everyw...|
|20113|Clinton, Sanders ...|New York Times|    James Poniewozik|16/04/2016|2016|    4|Thursday night, D...|
|20114|Yahoo�s Suitors U...|New York Times|Vindu Goel and Mi...|18/04/2016|2016|    4|SAN FRANCISCO  � ...|
+-----+--------------------+--------------+--------------------+----------+----+-----+--------------------+
only showing top 5 rows



In [72]:
val articlesfiltred=articles1.filter("month = 6").select("publication")
val articlesfiltred1=articles1.filter("year = 2016").groupBy("publication").count()
articlesfiltred1.show()
articlesfiltred.count()

+----------------+-----+
|     publication|count|
+----------------+-----+
|Business Insider|   14|
|             CNN| 4104|
|  New York Times| 3628|
|       Breitbart|16100|
+----------------+-----+



articlesfiltred = [publication: string]
articlesfiltred1 = [publication: string, count: bigint]


2209

In [41]:
//calcul statistiques
articles1.describe().show()
articles1.count()
schema = StructType([StructField("docid", StringType(), True),StructField("source", StringType(), True),StructField("url", StringType(), True),StructField("title", StringType(), True),StructField("summary", StringType(), True),StructField("text", StringType(), True),StructField("date", StringType(), True)])


articles1.printSchema

+-------+------------------+--------------------+--------------+--------------------+----------+------+------------------+--------------------+
|summary|             docid|               title|   publication|              author|      date|  year|             month|                text|
+-------+------------------+--------------------+--------------+--------------------+----------+------+------------------+--------------------+
|  count|             23846|               23846|         23846|               22219|     23846| 23846|             23846|               23846|
|   mean| 41977.41763817831|                null|          null|                null|      null|2016.0| 7.213662668791412|                null|
| stddev|10633.814549540011|                null|          null|                null|      null|   0.0|3.2824944658486532|                null|
|    min|             17283|"***LIVE At 5:30P...|     Breitbart|         A. O. Scott|01/01/2016|  2016|                 1|#AmericaWasNev

In [65]:
 val news: RDD[(Int, java.sql.Timestamp, String)] = news_xml.map(e => extractAll(e,textToExtract))
    val newsTitles: RDD[(Int, java.sql.Timestamp, String)] = news_xml.map(e => extractAll(e,"title"))
    val newsSummaries: RDD[(Int, java.sql.Timestamp, String)] = news_xml.map(e => extractAll(e,"text"))

nysk = MapPartitionsRDD[102] at map at <console>:134
nyskTitles = MapPartitionsRDD[103] at map at <console>:135
nyskSummaries = MapPartitionsRDD[104] at map at <console>:136


MapPartitionsRDD[104] at map at <console>:136

In [66]:
    val stopwords = sc.textFile("stopwords.txt").collect.toArray.toSet
    val stopwordsBroadcast = sc.broadcast(stopwords).value

stopwords = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...


Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...

In [67]:
   val lemmatizedWithDate = news.mapPartitions(iter => {
      val pipeline = com.cloudera.datascience.lsa.ParseWikipedia.createNLPPipeline();
      iter.map {
        case (docid, date, text) =>
          (docid.toString, date,
            com.cloudera.datascience.lsa.ParseWikipedia.plainTextToLemmas(text.toLowerCase.split("\\W+").mkString(" "), stopwordsBroadcast, pipeline))
        };
    })

lemmatizedWithDate = MapPartitionsRDD[107] at mapPartitions at <console>:133


MapPartitionsRDD[107] at mapPartitions at <console>:133

In [68]:
    val lemmatized = lemmatizedWithDate.map { case (docid, date, text) => (docid, text) }
    val numTerms = 1000;

lemmatized = MapPartitionsRDD[108] at map at <console>:131
numTerms = 1000


1000

In [ ]:
    val (termDocMatrix, termIds, docIds, idfs) = com.cloudera.datascience.lsa.ParseWikipedia.termDocumentMatrix(lemmatized, stopwordsBroadcast, numTerms, sc);
    termDocMatrix.cache();



In [ ]:
val termIdsTxT = termIds.map(l => l.toString.filter(c => c != '[' & c != ']'))

val outputtermIds= "data/termIds.txt"
sc.makeRDD(termIdsTxT.toList).saveAsTextFile("outputtermIds")

In [27]:
val mat = new RowMatrix(termDocMatrix)
val k = 10 // nombre de valeurs singuliÃ¨res Ã  garder
val svd = mat.computeSVD(k, computeU=true)
val projections = mat.multiply(svd.V)
val projectionsTxt = projections.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
// Delete the existing path, ignore any exceptions thrown if the path doesn't exist
val outputProjection = "data/projection_LSA.txt"

projectionsTxt.saveAsTextFile(outputProjection)

In [63]:
        val nbClusters = 10
        val nbIterations = 1000
        val runs = 10
        val clustering = KMeans.train(termDocMatrix, nbClusters, nbIterations, runs, "k-means||", 0)
        /*val outputClustering = "hdfs://head.local:9000/user/emeric/clusters"
        try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClustering), true) } 
        catch { case _ : Throwable => { } }
        clustering.save(sc, outputClustering)*/
        
        val classes = clustering.predict(termDocMatrix)
        val outputClasses = "data/classes_LSA.txt"
    
        classes.saveAsTextFile(outputClasses)

nbClusters = 10
nbIterations = 1000
runs = 10
clustering = org.apache.spark.mllib.clustering.KMeansModel@669cb1c2
classes = MapPartitionsRDD[146] at map at KMeansModel.scala:85
outputClasses = data/classes_LSA.txt


data/classes_LSA.txt

In [34]:
 
        val outputData = lemmatizedWithDate.zip(classes).map { case ((docid, date, title),cl) => (docid, date, cl) }.sortBy(_._2).map(l => l.toString.filter(c => c != '(' & c != ')'))
        val outputDataFile = "data/output_LSA.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputDataFile), true) } 
        //catch { case _ : Throwable => { } }
        outputData.saveAsSingleTextFile(outputDataFile)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma


outputData = MapPartitionsRDD[161] at map at <console>:196
outputDataFile = output/output_LSA.txt


output/output_LSA.txt

In [ ]:
       
        clustering.clusterCenters.foreach(clusterCenter => {
            val highest = clusterCenter.toArray.zipWithIndex.sortBy(-_._1).map(v => v._2).take(10)
            println("*****")
            highest.foreach { s => print( termIds(s) + "," ) }
            println ()
            }
       )

In [40]:
import org.apache.spark.mllib.feature.Word2VecModel
// lire le Word2VecModel
val w2vModel = Word2VecModel.load(sc, "w2vModel")

w2vModel = org.apache.spark.mllib.feature.Word2VecModel@78c028b5


<console>:6: error: Symbol 'type scala.AnyRef' is missing from the classpath.
This symbol is required by 'class org.apache.spark.sql.catalyst.QualifiedTableName'.
Make sure that type AnyRef is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'QualifiedTableName.class' was compiled against an incompatible version of scala.
  lazy val $print: String =  {
           ^


org.apache.spark.mllib.feature.Word2VecModel@78c028b5

In [ ]:
val vectors = w2vModel.getVectors.mapValues(vv => org.apache.spark.mllib.linalg.Vectors.dense(vv.map(_.toDouble))).map(identity)
        // transmettre la map aux noeuds de calcul
val bVectors = sc.broadcast(vectors)

In [37]:

        val idTerms = termIds.map(_.swap)

        val pairs = termDocMatrix.zip(lemmatized);

        var w2vecRepr = pairs.map({ case (row, (docid, lemmas)) => 
          var vSum = Vectors.zeros(100)
          var totalWeight: Double = 0
          val words = lemmas.toSet
          words.foreach { word =>
               val colId = idTerms.get(word);
               if (colId != None) {
                   var weight = row(colId.get);
                   if (! weightTfIdf) {
                       weight = 1
                   }
                   val wvec = bVectors.value.get(word)
                   if (wvec != None) {          
                       vSum = add(scalarMultiply(weight, wvec.get), vSum)
                       totalWeight += weight
                   }
               }
          }
          if (totalWeight > 0) {
               vSum = scalarMultiply(1.0 / totalWeight, vSum)
          }
          else {
               vSum = Vectors.zeros(100);
          }
          (docid, vSum)
        })/*.filter(vec => Vectors.norm(vec._2, 1.0) > 0.0)*/.persist() 

Name: Unknown Error
Message: lastException: Throwable = null
<console>:216: error: not found: value bVectors2
                          val wvec = bVectors2.value.get(word)
                                     ^

StackTrace: 

In [38]:
val matRDD = w2vecRepr.map{v => v._2}.cache()
        val mat = new RowMatrix(matRDD)
        val matrixTxt = mat.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
        // Delete the existing path, ignore any exceptions thrown if the path doesn't exist
        val outputMatrix = "data/matrice_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputMatrix), true) } 
        //catch { case _ : Throwable => { } }
        matrixTxt.saveAsTextFile(outputMatrix)

Name: Unknown Error
Message: <console>:194: error: not found: value w2vecRepr
       val matRDD = w2vecRepr.map{v => v._2}.cache()
                    ^

StackTrace: 

In [ ]:
        val centRed = new StandardScaler(withMean = true, withStd = true).fit(matRDD)
        val matCR: RowMatrix = new RowMatrix(centRed.transform(matRDD))

        val matCompPrinc = matCR.computePrincipalComponents(10)
        val projections = matCR.multiply(matCompPrinc)
        //val matSummary = projections.computeColumnSummaryStatistics()
        val projectionsTxt = projections.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
        // Delete the existing path, ignore any exceptions thrown if the path doesn't exist
        val outputProjection = "data/projection_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputProjection), true) } 
        //catch { case _ : Throwable => { } }
        projectionsTxt.saveAsTextFile(outputProjection)

In [ ]:
     val nbClusters = 10
        val nbIterations = 1000
        val runs = 10
        val clustering = KMeans.train(matRDD, nbClusters, nbIterations, runs, "k-means||", 0)
        /*val outputClustering = "hdfs://head.local:9000/user/emeric/clusters"
        try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClustering), true) } 
        catch { case _ : Throwable => { } }
        clustering.save(sc, outputClustering)*/
        
        val classes = clustering.predict(matRDD)
        val outputClasses = "data/classes_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClasses), true) } 
        //catch { case _ : Throwable => { } }
        classes.saveAsTextFile(outputClasses)

In [ ]:
        val outputData = lemmatizedWithDate.zip(classes).map { case ((docid, date, title),cl) => (docid, date, cl) }.sortBy(_._2).map(l => l.toString.filter(c => c != '(' & c != ')'))
        val outputDataFile = "data/output_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputDataFile), true) } 
        //catch { case _ : Throwable => { } }
        outputData.saveAsTextFile(outputDataFile)

In [ ]:
   clustering.clusterCenters.foreach(clusterCenter => {
            val nearest = w2vecRepr.map{v => (v._1, Vectors.sqdist(v._2,clusterCenter))}.sortBy(_._2).map{ case (id, dist) => id }.take(10).toSet
            println("*****")
            pairs.filter{ case (row, (docid, lemmas)) => nearest contains docid }.foreach {  case (row, (docid, lemmas)) => {
                val id = row.toArray.zipWithIndex.sortBy(- _._1).take(5).map(_._2)
                id.foreach { s => print( termIds(s) + "," ) }
                println ()
                }
            }

In [ ]:
import org.apache.spark.mllib.feature.Word2VecModel
// lire le Word2VecModel
val w2vModel = Word2VecModel.load(sc, "w2vModelwiki8")